1. Importing libraries 

In [778]:
import re, numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns, os 

Setting display options

In [779]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

2. Reading the data 

In [780]:
PROJECT_DIR  = "/Users/abhisheksaurav/Desktop/real_estate_project/"
DATA_DIR = "Data Collection/Data/City/chandigarh/cleaned_data"

In [781]:
def get_data(name):
    file_name = "{}.csv".format(name)
    file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)
    return pd.read_csv(file_path)

In [782]:
df = get_data("gurgaon_properties_cleaned_v1")

In [783]:
df.head(1)

,property type,society,sector,price,price_per_sq_ft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,floorNum,facing,agePossession,nearbyLocations,furnishDetails,features
0,flat,ats kocoon,sector 109,1.7,9742.0,1745.0,Super Built up area 1745(162.12 sq.m.)Built Up area: 1550 sq.ft. (144 sq.m.),3,3,3,not available,22.0,east,1 to 5 Year Old,"['Jagdish Super Market', 'Dwarka', 'Euro International School', 'Gyaananda School', 'SCR Model School', 'Radha Krishan Mandir', 'ICICI BANK ATM, Annapurna MKT', 'ESIC Dispensary', 'IGIA Airport', 'Canara Bank New Palam Vihar', 'Gurgaon Gramin Bank', 'Daultabad Stadium', 'Bharat Petroleum Shree Shyam Filling', 'The Club, International City', 'Conscient One']","['1 Modular Kitchen', 'No AC', 'No Bed', 'No Chimney', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Fan', 'No Geyser', 'No Light', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Wardrobe', 'No Washing Machine', 'No Water Purifier']","['Security / Fire Alarm', 'Intercom Facility', 'Lift(s)', 'Maintenance Staff', 'Swimming Pool', 'Water Storage', 'Park', 'Visitor Parking', 'Internet/wi-fi connectivity', 'Shopping Centre', 'Fitness Centre / GYM', 'Club house / Community Center']"


4. Detailed Analysis

4.1 areaWithType

In [784]:
(df
 .sample(5)[['price','area', 'areaWithType']]
)

,price,area,areaWithType
2234,0.28,498.0,Carpet area: 489 (45.43 sq.m.)
2406,0.75,1342.0,Super Built up area 1342(124.68 sq.m.)Built Up area: 1180 sq.ft. (109.63 sq.m.)Carpet area: 1050 sq.ft. (97.55 sq.m.)
1071,2.50,3194.0,Plot area 3194(296.73 sq.m.)
3150,0.45,531.0,Carpet area: 531 (49.33 sq.m.)
3443,1.40,1023.0,Super Built up area 1650(153.29 sq.m.)Carpet area: 1022.58 sq.ft. (95 sq.m.)


In [785]:
# function to extract the Super built up area 

def get_super_built_up_area(text):
    match = re.search(r'Super Built up area (\d+\.?\d*)', text)
    if match:
        return float(match.group(1))
    return None

In [786]:
# function to extract the built up area or carpet area 

def get_area(text, area_type):
    match = (re
             .search(area_type + r'\s*:\s*(\d+\.?\d*)', 
                     text)
    )
    if match:
        return float(match
                     .group(1))
    return None

In [787]:
# this function checks if the area is in sq.m and converts to square feet if needed 

def convert_to_sqft(text, area_value):
    if area_value is None:
        return None
    match = re.search(r'{} \((\d+\.?\d*) sq.m.\)'.format(area_value), text)
    if match:
        sq_m_value = float(match.group(1))
        return sq_m_value * 10.7639  # conversion factor from sq.m. to sqft
    return area_value


In [788]:
# extracting super built up area and converting it into sq_ft if needed 

df['super_built_up_area'] = (df
                             .areaWithType
                             .apply(get_super_built_up_area)
)
df['super_built_up_area'] = (df
                             .apply(lambda x: convert_to_sqft(x['areaWithType'], 
                                                              x['super_built_up_area']), 
                                                              axis=1)
)

In [789]:
# extracting built up area and converting it if needed 

df['built_up_area'] = (df
                       .areaWithType
                       .apply(lambda x: get_area(x, 'Built Up area')))

df['built_up_area'] = (df
                       .apply(lambda x: convert_to_sqft(x['areaWithType'], 
                                              x['built_up_area']), axis = 1))

In [790]:
# extracting carpet area and converting it if needed 

df['carpet_area'] = (df 
                     .areaWithType 
                     .apply(lambda x: get_area(x, "Carpet area")))

df["carpet_area"] = (df
                     .apply(lambda x: convert_to_sqft(x['areaWithType'], 
                                                      x['carpet_area']), axis= 1)
                        )

In [791]:
df[['price', 'property type', 'price_per_sq_ft', 'area', 'areaWithType','super_built_up_area', 'built_up_area', 'carpet_area']].sample(5)

,price,property type,price_per_sq_ft,area,areaWithType,super_built_up_area,built_up_area,carpet_area
2842,2.45,flat,11887.0,2061.0,Super Built up area 2061(191.47 sq.m.),2061.0,NaN,NaN
2817,1.10,flat,8800.0,1250.0,Carpet area: 1250 (116.13 sq.m.),NaN,NaN,1250.0
720,6.00,house,21097.0,2844.0,Plot area 316(264.22 sq.m.),NaN,NaN,NaN
3068,0.36,flat,5142.0,700.0,Built Up area: 700 (65.03 sq.m.),NaN,700.0,NaN
1619,0.75,flat,3067.0,2445.0,Carpet area: 2445 (227.15 sq.m.),NaN,NaN,2445.0


In [792]:
(
    df 
    .duplicated() 
    .sum()
)

np.int64(125)

- now there are 125 duplicate rows. 

In [793]:
# checking the rows in which we don't have 'Super built up area', ' Built up area' or 'carpet area' 

df[~(df['super_built_up_area'].isnull() | df['built_up_area'].isnull() | df['carpet_area'].isnull())].sample(3)

,property type,society,sector,price,price_per_sq_ft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,floorNum,facing,agePossession,nearbyLocations,furnishDetails,features,super_built_up_area,built_up_area,carpet_area
2251,flat,raheja teachers apartments,sector 31,1.2,7272.0,1650.0,Super Built up area 1650(153.29 sq.m.)Built Up area: 1450 sq.ft. (134.71 sq.m.)Carpet area: 1150 sq.ft. (106.84 sq.m.),3,2,2,others,1.0,east,5 to 10 Year Old,"['Hanuman Mandir', 'Icici bank ATM', 'State bank of india ATM', 'Axis bank ATM', 'Hdfc bank ATM', 'State bank of india ATM', 'Icici ATM', 'Citi bank ATM', 'Axis bank ATM', 'Hdfc ATM', 'Axis bank ATM', 'Shivam Hospital Gurgaon', 'Ahmed Hospital Multi Speciality', 'Dispencery', 'Dayal Eye & Maternity Centre', 'Bansal Medicare and Maternity Centre', 'Pushpanjali Hospital', 'Pushpanjali Hospital Gurgaon', 'Kalyani Hospital Gurgaon', 'Medanta', 'Aarvy Hospital', 'Centre For Sight Gurgaon Sector 29', 'Saraswati Hospital Gurgaon', 'Gardian Pharmacy', 'City Medical', 'Gardian Pharmacy', 'IBP Petrol Pump', 'Indian Oil', 'Indian Oil', 'Hdfc bank', 'State bank of india sbi', 'Punjab national bank', 'Om Sweets', 'Fast Food', 'Fast food', 'Bar and restaurant', 'Cafe Coffee Day', 'Darbar', '32nd Milestone', 'Raj Restaurant', 'Dhabba', 'KFC', 'CR Model Public School', 'Salvan Public School', 'St. Angels Jr', 'govt sec school', 'Manav Rachna School', 'Manav Rachna Swimming Pool', 'District library gurgaon']","['1 Water Purifier', '5 Fan', '1 Exhaust Fan', '1 Dining Table', '2 Geyser', '1 Stove', '20 Light', '4 AC', '1 Chimney', '7 Curtains', '1 Modular Kitchen', '13 Wardrobe', '1 Microwave', 'No Bed', 'No Fridge', 'No Sofa', 'No TV', 'No Washing Machine']","['Security / Fire Alarm', 'Feng Shui / Vaastu Compliant', 'Private Garden / Terrace', 'Intercom Facility', 'Lift(s)', 'Centrally Air Conditioned', 'Water purifier', 'Maintenance Staff', 'Water Storage', 'Bank Attached Property', 'Piped-gas', 'Visitor Parking', 'Park', 'Security Personnel', 'Internet/wi-fi connectivity', 'Recently Renovated', 'Natural Light', 'Waste Disposal', 'Rain Water Harvesting', 'Club house / Community Center', 'Water softening plant']",1650.0,1450.0,1150.0
377,flat,orris carnation residency,sector 85,0.9,4531.0,1986.0,Super Built up area 1975(183.48 sq.m.)Built Up area: 1575 sq.ft. (146.32 sq.m.)Carpet area: 1435 sq.ft. (133.32 sq.m.),4,4,3,not available,8.0,east,5 to 10 Year Old,"['Omaxe Celebration Mall within reach', 'Nearby Lotus Valley International School', 'The Shri Ram School Aravali', 'Close to ARC Multi Specialty Hospital', 'Proximity to Maxe Healthcare Hospital', 'Close to Medanta, The Medicity', 'Walking distance to Children Park', 'Hyatt Regency in the neighborhood']","['3 Wardrobe', '6 Fan', '1 Exhaust Fan', '2 Geyser', '12 Light', '1 Modular Kitchen', '1 Chimney', 'No AC', 'No Bed', 'No Curtains', 'No Dining Table', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Washing Machine', 'No Water Purifier']","['Feng Shui / Vaastu Compliant', 'Intercom Facility', 'Lift(s)', 'Maintenance Staff', 'Water Storage', 'Park', 'Visitor Parking']",1975.0,1575.0,1435.0
3858,flat,la vida by tata housing,sector 113,3.0,14285.0,2100.0,Super Built up area 2691(250 sq.m.)Built Up area: 2460 sq.ft. (228.54 sq.m.)Carpet area: 2100 sq.ft. (195.1 sq.m.),3,3,3+,servant room,3.0,east,1 to 5 Year Old,"['Dwarka Sector 21', 'Pacific D21 Mall', 'Bajghera Road', 'Dwarka Expy', 'Euro International School, Sector- 109.', 'The NorthCap University', ""Rion's Hospital"", 'Indira Gandhi Intl Airport', 'Bijwasan Railway Station']","['1 AC', 'No Bed', 'No Chimney', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Fan', 'No Geyser', 'No Modular Kitchen', 'No Light', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Wardrobe', 'No Washing Machine', 'No Water Purifier']","['Centrally Air Conditioned', 'Water purifier', 'Security / Fire Alarm', 'Feng Shui / Vaastu Compliant', 'Private Garden / Terrace', 'Intercom Fac

In [794]:
df[~(df['super_built_up_area'].isnull() | df['built_up_area'].isnull() | df['carpet_area'].isnull())].shape

(537, 20)

- there are 537 such rows where we have he all the values of super built up area, built up area and carpet area. 
- out of 3942 records for various flats there are only 537 rows in which we have all the values related to built up area 
- this means that 3405 rows in which atleast one of these values is missing, we will use these 537 rows to fill those values 

In [795]:
# checking the rows in which we have plot area, for these rows the super_built_up_area, built_up_area, carpet_area will be missing 

(
    df[df 
    .areaWithType 
    .str 
    .contains("Plot")][['price', 'property type', 'price_per_sq_ft', 'area', 'areaWithType','super_built_up_area', 'built_up_area', 'carpet_area']]
    #.sample(5)
    .shape

)

(749, 8)

- there are 748 rows in which we have plot area. 

In [796]:
# checking for count of missing values in the 'super_built_up_area', 'built_up_area' and 'carpet_area'

df[['super_built_up_area', 'built_up_area', 'carpet_area']].isnull().sum()

super_built_up_area    2008
built_up_area          2713
carpet_area            1941
dtype: int64

Out of 3942 rows - 
- super built up area has 2008 missing values
- built up area has 2713 missing values 
- carpet area has 1941 mising values 

In [797]:
# extracting and converting to dataframe for those rows in which 'super_built_up_area', 'built_up_area' and 'carpet_area' is missing 

all_nan_df = (
    df[(df 
    .super_built_up_area 
    .isnull()) & ( 
        df 
        .built_up_area 
        .isnull() & (
            df 
            .carpet_area 
            .isnull()
        )
    )][['price', 'property type', 'area', 'areaWithType','super_built_up_area', 'built_up_area', 'carpet_area']] 
    # .sample(3)
    # .shape
)

In [798]:
(
    all_nan_df
    .head()
)

,price,property type,area,areaWithType,super_built_up_area,built_up_area,carpet_area
6,3.40,house,1530.0,Plot area 170(142.14 sq.m.),NaN,NaN,NaN
7,3.05,house,2430.0,Plot area 270(225.75 sq.m.),NaN,NaN,NaN
13,14.00,house,3510.0,Plot area 390(326.09 sq.m.),NaN,NaN,NaN
14,9.50,house,546.0,Plot area 546(50.73 sq.m.),NaN,NaN,NaN
15,8.00,house,4518.0,Plot area 502(419.74 sq.m.),NaN,NaN,NaN


- super_built_up_area	built_up_area	carpet_area are missing in only those rows where we have the record of the independent houses and for those records we have the plot area 
- our next goal is to extract those plot areas for the independent houses

In [799]:
# storing the index values for the records where the super_built_up_area	built_up_area	carpet_area is missing 

all_nan_df_index = (
    df[(df 
    .super_built_up_area 
    .isnull()) & ( 
        df 
        .built_up_area 
        .isnull() & (
            df 
            .carpet_area 
            .isnull()
        )
    )][['price', 'property type', 'area', 'areaWithType','super_built_up_area', 'built_up_area', 'carpet_area']] 
    .index
)

In [800]:
# creating function for extracting the plot area 

def extract_plot_area(area_with_type):
    match = re.search(r"Plot area (\d+\.?\d*)", area_with_type)
    if match : 
        return float(match.group(1))
    else: 
        return None 

In [801]:
# here we are extracting the plot area and putting it into the built up area as for the independent house case the plot area is quite similar to the built up area

all_nan_df['built_up_area'] = (
    all_nan_df
    .areaWithType
    .apply(extract_plot_area)
)

In [802]:
all_nan_df.sample(5)

,price,property type,area,areaWithType,super_built_up_area,built_up_area,carpet_area
2583,4.25,house,1350.0,Plot area 150(125.42 sq.m.),NaN,150.0,NaN
2764,0.37,house,360.0,Plot area 360(33.45 sq.m.),NaN,360.0,NaN
140,3.50,house,1722.0,Plot area 160,NaN,160.0,NaN
1654,10.55,house,2700.0,Plot area 300(250.84 sq.m.),NaN,300.0,NaN
1692,0.60,house,720.0,Plot area 720(66.89 sq.m.),NaN,720.0,NaN


- here, there is some discrepancy in the plot area as some are in square feet,  square yard and square meter. 
- to fix this issue we need to convert those plot areas who are in square yards and square meter to square feets
- we will use the following formula 
    - square feet = 9 * square yard 
    - square feet = 10.7 * square meter 

In [803]:
# scale conversion formula function 

def convert_scale(row):
    if np.isnan(row['area']) or np.isnan(row['built_up_area']):
        return row['built_up_area']
    else: 
        if round(row['area']/ row['built_up_area']) == 9.0: 
            return row['built_up_area'] * 9 
        elif round(row['area']/row['built_up_area']) == 11.0: 
            return row['built_up_area'] * 10.7 
        else: 
            return row['built_up_area']

In [804]:
all_nan_df.loc[2487][['price',	'property type',	'area',	'areaWithType',	'super_built_up_area',	'built_up_area',	'carpet_area']]

price                                       0.75
property type                              house
area                                       100.0
areaWithType           Plot area 100(9.29 sq.m.)
super_built_up_area                          NaN
built_up_area                              100.0
carpet_area                                  NaN
Name: 2487, dtype: object

In [805]:
# converting the scale 

all_nan_df['built_up_area'] = (
    all_nan_df 
    .apply(convert_scale, axis = 1)
)

In [806]:
all_nan_df.sample(5)

,price,property type,area,areaWithType,super_built_up_area,built_up_area,carpet_area
2016,3.50,house,1449.0,Plot area 161(134.62 sq.m.),NaN,1449.0,NaN
859,6.50,house,5400.0,Plot area 5400(501.68 sq.m.),NaN,5400.0,NaN
2043,0.95,house,1070.0,Plot area 1070(99.41 sq.m.),NaN,1070.0,NaN
1109,4.95,house,2403.0,Plot area 267(223.25 sq.m.),NaN,2403.0,NaN
267,11.95,house,2745.0,Plot area 305(255.02 sq.m.),NaN,2745.0,NaN


In [807]:
# updating the original dataframe with these values 

df.update(all_nan_df)

In [808]:
df[['super_built_up_area', 'built_up_area', 'carpet_area']].isnull().sum()

super_built_up_area    2008
built_up_area          2113
carpet_area            1941
dtype: int64

- by using the above operations we significantly reduced the number of missing values in the built up area. 
- earlier the missing values in the built up area was around 2713 which has now reduced to 2113. 

In [809]:
# current state of our dataframe

df.head()

,property type,society,sector,price,price_per_sq_ft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,floorNum,facing,agePossession,nearbyLocations,furnishDetails,features,super_built_up_area,built_up_area,carpet_area
0,flat,ats kocoon,sector 109,1.70,9742.0,1745.0,Super Built up area 1745(162.12 sq.m.)Built Up area: 1550 sq.ft. (144 sq.m.),3,3,3,not available,22.0,east,1 to 5 Year Old,"['Jagdish Super Market', 'Dwarka', 'Euro International School', 'Gyaananda School', 'SCR Model School', 'Radha Krishan Mandir', 'ICICI BANK ATM, Annapurna MKT', 'ESIC Dispensary', 'IGIA Airport', 'Canara Bank New Palam Vihar', 'Gurgaon Gramin Bank', 'Daultabad Stadium', 'Bharat Petroleum Shree Shyam Filling', 'The Club, International City', 'Conscient One']","['1 Modular Kitchen', 'No AC', 'No Bed', 'No Chimney', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Fan', 'No Geyser', 'No Light', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Wardrobe', 'No Washing Machine', 'No Water Purifier']","['Security / Fire Alarm', 'Intercom Facility', 'Lift(s)', 'Maintenance Staff', 'Swimming Pool', 'Water Storage', 'Park', 'Visitor Parking', 'Internet/wi-fi connectivity', 'Shopping Centre', 'Fitness Centre / GYM', 'Club house / Community Center']",1745.0,1550.0,NaN
1,house,independent,sector 36,0.66,7168.0,921.0,Built Up area: 823 (76.46 sq.m.),2,2,3,not available,4.0,NaN,0 to 1 Year Old,NaN,NaN,NaN,NaN,823.0,NaN
2,flat,gls arawali homes,sector 163,0.28,4938.0,567.0,Super Built up area 567(52.68 sq.m.)Built Up area: 500 sq.ft. (46.45 sq.m.)Carpet area: 467 sq.ft. (43.39 sq.m.),2,2,2,not available,3.0,east,1 to 5 Year Old,"['Huda City Centre', 'Golf Course Road', 'Delhi-Mumbai Expressway', 'KMP Expressway', 'Rajiv Chowk', 'IGI Airport']","['3 Fan', '1 Exhaust Fan', '4 Light', '2 Curtains', '1 Modular Kitchen', '2 Wardrobe', 'No AC', 'No Bed', 'No Chimney', 'No Dining Table', 'No Geyser', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Washing Machine', 'No Water Purifier']","['Feng Shui / Vaastu Compliant', 'Security / Fire Alarm', 'Intercom Facility', 'Lift(s)', 'Maintenance Staff', 'Water Storage', 'Park']",567.0,500.0,467.0
3,flat,breez global heights,sector 36,0.24,5825.0,412.0,Carpet area: 412 (38.28 sq.m.),1,1,1,not available,8.0,na,0 to 1 Year Old,"['S.R.S. Hospital and Critical Care Unit', 'Shri Balaji Hospital and Trauma Center', 'Chandna Dental Surgery Orthodontic and Implant Centre', 'The Muskan Dental Clinic', 'Dental Xpert Dental Clinic', 'Ayushman Hospital And Trauma Centre', 'Yadav Hospital Gurgoan', 'Kamla Hospital Gurgaon', 'Harshila Dental Clinic', 'Clove Dental', 'Vaishnavi Nursing Home', 'Petrol Pump IBP', 'Petrol Pump Indian Oil', 'Boxer Fuel Point', 'Haldiram']","['1 Wardrobe', '2 Fan', '1 Exhaust Fan', '5 Light', '1 Modular Kitchen', '1 Curtains', 'No AC', 'No Bed', 'No Chimney', 'No Dining Table', 'No Geyser', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Washing Machine', 'No Water Purifier']","['Feng Shui / Vaastu Compliant', 'Intercom Facility', 'Lift(s)', 'Maintenance Staff', 'Park', 'Security Personnel', 'Shopping Centre', 'Fitness Centre / GYM', 'Club house / Community Center', 'Rain Water Harvesting']",NaN,NaN,412.0
4,house,rwa sector 46,sector 46,4.50,24390.0,1845.0,Plot area 204(170.57 sq.m.)Built Up area: 206 sq.yards (172.24 sq.m.)Carpet area: 205 sq.yards (171.41 sq.m.),6,6,3+,store room,3.0,North-East,5 to 10 Year Old,"['Axis bank ATM', 'State bank of india ATM', 'Icici bank ATM', 'Icici ATM', 'Standard chartered ATM', 'Hdfc bank ATM', 'Citi bank ATM', 'Axis bank ATM', 'Hdfc ATM', 'Standard chartered ATM', 'Dispencery', 'Sukhmani Hospital Pvt. Ltd', ""DR AKRAM JAWED'S THE UPPER LIMB CLINIC"", 'Medanta', 'Meher Clinic', 'Park Hospital Gurgaon', 'Vishesh Dental', 'Bones Clinic - Orthopaedics', 'Neelkanth Health Care', 'Shivam Hospital Gurgaon', 'Samvit Health Care', 'Pernod Ricard Charitable Dispensary', 'Gardian Pharmacy', 'City Medical', 'Gardian Pharma

4.2 additional rooms

In [810]:
(
    df 
    .additionalRoom
    .value_counts()
)

additionalRoom
not available                                    1681
servant room                                      709
study room                                        251
others                                            236
pooja room                                        170
store room                                        107
study room,servant room                           100
pooja room,servant room                            83
pooja room,study room,servant room,store room      71
servant room,others                                59
pooja room,study room,servant room                 57
pooja room,study room,servant room,others          56
servant room,pooja room                            38
servant room,store room                            34
study room,others                                  30
pooja room,study room                              24
pooja room,others                                  17
pooja room,store room                              17
servant room,

- since there are multiple combinations of the extra rooms available but broadly these rooms are divided into 5 categories
    - servent room 
    - pooja room 
    - others 
    - store room 
    - study room 
- now we will create 5 extra rooms based on these columns which is a kind of one hot encoding for thesee values 

In [811]:
# list of new cols to be created 
new_cols = ['study room', 'servant room', 'store room', 'pooja room', 'others']

# populating the new col based on the list 
for col in new_cols: 
    df[col] = (
        df['additionalRoom']
        .str 
        .contains(col) 
        .astype(int)
    )


In [812]:
(
    df 
    .sample(5)[['additionalRoom','study room', 'servant room', 'store room', 'pooja room', 'others']]
)

,additionalRoom,study room,servant room,store room,pooja room,others
3470,not available,0,0,0,0,0
128,"pooja room,study room,servant room",1,1,0,1,0
2661,not available,0,0,0,0,0
2026,study room,1,0,0,0,0
1401,not available,0,0,0,0,0


4.4 agePossession 

In [813]:
( 
    df 
    .agePossession
    .value_counts()
)

agePossession
1 to 5 Year Old       1702
5 to 10 Year Old       604
0 to 1 Year Old        550
undefined              358
10+ Year Old           345
Under Construction      92
Within 6 months         70
Within 3 months         26
Dec 2023                22
By 2023                 19
By 2024                 17
Dec 2024                16
Mar 2024                14
Oct 2024                 8
Jan 2024                 8
Aug 2023                 7
Dec 2025                 7
Jun 2024                 7
Aug 2024                 5
Nov 2023                 5
Jul 2024                 4
Sep 2023                 4
Jan 2025                 4
Oct 2023                 4
By 2025                  4
May 2024                 3
Feb 2024                 3
Nov 2024                 3
Jun 2027                 2
Mar 2025                 2
Jul 2027                 2
Oct 2025                 2
Jan 2026                 2
Dec 2026                 2
Jul 2025                 2
Sep 2025                 2
Apr 2026      

In [814]:
def cateorize_age_possession(value):

    categories = {
        "New Property": ["0 to 1 Year Old", "Within 6 months", "Within 3 months"],
        "Relatively New": ["1 to 5 Year Old"],
        "Moderately Old": ["5 to 10 Year Old"],
        "Old Property": ["10+ Year Old"],
        "Under Construction": ["Under Construction", "By"]}
    
    if pd.isna(value):
        return 'undefined'

    for category, keywords in categories.items(): 
        if any(keyword in value for keyword in keywords):
            return category
    
    try: 
        if int(value.split(" ")[-1]):
            return "Under Construction" 
    except ValueError: 
        pass 
 
    return "undefined"
        

In [815]:
df['agePossession'] = (
    df 
    .agePossession 
    .apply(cateorize_age_possession)
)

In [816]:
(
    df 
    .agePossession 
    .value_counts()
)

agePossession
Relatively New        1702
New Property           646
Moderately Old         604
undefined              359
Old Property           345
Under Construction     286
Name: count, dtype: int64

- the number of columns in the agePossession category has significantly reduced now. 
- it's reduced to 5 categories 

4.5 furnish details 

In [817]:
(
    df[['furnishDetails','features']]
    .sample(5)
)

,furnishDetails,features
2869,NaN,"['Feng Shui / Vaastu Compliant', 'Intercom Facility', 'Lift(s)', 'Maintenance Staff', 'Park']"
1870,NaN,"['Water purifier', 'Security / Fire Alarm', 'Feng Shui / Vaastu Compliant', 'Private Garden / Terrace', 'Intercom Facility', 'Lift(s)', 'Maintenance Staff', 'Water Storage', 'Separate entry for servant room', 'No open drainage around', 'Bank Attached Property', 'Piped-gas', 'Internet/wi-fi connectivity', 'Recently Renovated', 'Visitor Parking', 'Swimming Pool', 'Park', 'Security Personnel', 'Natural Light', 'Airy Rooms', 'Spacious Interiors', 'Low Density Society', 'Waste Disposal', 'Rain Water Harvesting', 'Water softening plant', 'Shopping Centre', 'Fitness Centre / GYM', 'Club house / Community Center']"
2640,"['1 Modular Kitchen', 'No AC', 'No Bed', 'No Chimney', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Fan', 'No Geyser', 'No Light', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Wardrobe', 'No Washing Machine', 'No Water Purifier']","['Power Back-up', 'Feng Shui / Vaastu Compliant', 'Intercom Facility', 'Lift(s)', 'Maintenance Staff', 'Piped-gas', 'Visitor Parking', 'Swimming Pool', 'Park', 'Security Personnel', 'Natural Light', 'Airy Rooms', 'Low Density Society', 'Shopping Centre', 'Fitness Centre / GYM', 'Waste Disposal', 'Rain Water Harvesting', 'Club house / Community Center']"
1900,"['3 Fan', '6 Light', 'No AC', 'No Bed', 'No Chimney', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Geyser', 'No Modular Kitchen', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Wardrobe', 'No Washing Machine', 'No Water Purifier']","['Power Back-up', 'Lift(s)', 'Park', 'Shopping Centre']"
2915,"['5 Fan', '1 Geyser', '5 Light', '4 AC', '1 Modular Kitchen', '1 Chimney', 'No Bed', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Wardrobe', 'No Washing Machine', 'No Water Purifier']","['Intercom Facility', 'Lift(s)', 'Swimming Pool', 'Park', 'Piped-gas', 'Shopping Centre', 'Fitness Centre / GYM', 'Club house / Community Center', 'Rain Water Harvesting']"


In [823]:

all_furnishings = []
for details in df['furnishDetails'].dropna():
    furnishings= (details.replace("[", '').replace(']', '').replace("'", "").split(","))
    all_furnishings.extend(furnishings)

In [852]:
unique_furnishings = list(set(all_furnishings))

In [854]:
unique_furnishings

['',
 ' 80 Light',
 ' No TV',
 '1 Bed',
 ' 2 Fan',
 '7 Light',
 ' 8 Wardrobe',
 ' 14 Fan',
 ' No Geyser',
 ' No Chimney',
 ' No Water Purifier',
 ' 13 AC',
 ' 19 Light',
 '6 Wardrobe',
 '1 Microwave',
 ' 1 Exhaust Fan',
 ' 3 Microwave',
 ' 16 Curtains',
 ' 8 Fan',
 '5 AC',
 ' 3 Curtains',
 ' 15 Wardrobe',
 ' 100 Light',
 ' 2 TV',
 ' 30 Light',
 '21 Light',
 ' 13 Wardrobe',
 '12 Fan',
 ' 1 Geyser',
 ' 1 Wardrobe',
 '1 Dining Table',
 ' 14 Light',
 ' 16 AC',
 ' 1 Dining Table',
 ' 2 Curtains',
 ' 21 Fan',
 '14 Bed',
 ' 16 Fan',
 ' 1 Curtains',
 ' 20 Fan',
 '22 Wardrobe',
 ' 55 Light',
 ' 9 Light',
 ' 13 Light',
 '9 Fan',
 '11 Wardrobe',
 ' 56 Light',
 '10 Bed',
 ' No Sofa',
 '2 Geyser',
 ' 18 AC',
 ' 24 Fan',
 '2 Wardrobe',
 ' 36 Light',
 ' 5 TV',
 ' 14 Wardrobe',
 ' 11 Bed',
 '3 AC',
 ' No Microwave',
 ' 70 Light',
 ' 4 AC',
 ' 10 Fan',
 ' 8 Curtains',
 ' 14 TV',
 ' No Stove',
 ' 2 Chimney',
 '26 Fan',
 ' 27 Light',
 ' 94 Light',
 ' 18 Chimney',
 '1 AC',
 ' 1 Light',
 '15 Light',
 ' 4 L